After some tweaking to the lambda api calls, plus based on the experiments I got from v1, now try v2 of analysis

In [1]:
import numpy as np
import pandas as pd
import pymongo
from pymongo import MongoClient
client = MongoClient()

from pprint import pprint
from bson.son import SON
#for geo parsing
import json
from shapely.geometry import shape, Point
import geopandas as gpd
from shapely.ops import cascaded_union

In [4]:
import datetime

In [2]:
#db setup-- the posts collection is the only one for stashing everything returned from that api endpoint
#note to self-- use mongoDatabase2
db = client['instagram-london']
coll = db.posts

In [20]:
coll.find_one({'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}})

{'_id': ObjectId('57d885b3d285908f0e11b78e'),
 'attribution': None,
 'caption': {'created_time': '1473706051',
  'from': {'full_name': 'Bec | Rightupmystreet',
   'id': '43060713',
   'profile_picture': 'https://scontent.cdninstagram.com/t51.2885-19/s150x150/13402212_659074237581639_631285985_a.jpg',
   'username': 'rightupmystreet'},
  'id': '17852498551109312',
  'text': 'Mackerel starter the Talbot in Ripley, Surrey 🍴🍷'},
 'comments': {'count': 1},
 'created_time': '1473706051',
 'filter': 'Normal',
 'id': '1337865782138595757_43060713',
 'images': {'low_resolution': {'height': 320,
   'url': 'https://scontent.cdninstagram.com/t51.2885-15/s320x320/e35/14240450_1160777360648826_2049808876_n.jpg?ig_cache_key=MTMzNzg2NTc4MjEzODU5NTc1Nw%3D%3D.2',
   'width': 320},
  'standard_resolution': {'height': 640,
   'url': 'https://scontent.cdninstagram.com/t51.2885-15/s640x640/sh0.08/e35/14240450_1160777360648826_2049808876_n.jpg?ig_cache_key=MTMzNzg2NTc4MjEzODU5NTc1Nw%3D%3D.2',
   'width': 640

In [21]:
#since I changed the range of the lat lngs returned from the lambda function, what I want is all the data after ~ 4th Sept 2016
rawInstagramPosts = coll.find({'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}})

In [23]:
uniqueLocationsCursor = db.posts.aggregate([
        {"$match": {'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}}},
        {"$group": { "_id": {"locationid": "$location.id","lat": "$location.latitude", "lng": "$location.longitude" }, 
                    "count": { "$sum": 1 }  }}
    ]);

uniqueLocations = list(uniqueLocationsCursor)

In [26]:
len(uniqueLocations)

41566

In [27]:
#now use Geopandas to parse the locations into the london census output areas
londonGeoDF = gpd.read_file('OA_2011_BGC_london.json')
#now trim the londonGeo stuff to may it smaller... hopefully this will make any op faster
londonDFSubset = londonGeoDF.ix[:,['OA11CD', 'POPDEN', 'WD11NM_BF', 'geometry']]

In [29]:
#using shapely, construct a geodataframe from the uniqueLocations
postLocationpoints = [Point(x['_id']['lng'], x['_id']['lat']) for x in uniqueLocations]
locationsDF = pd.DataFrame({'geometry': postLocationpoints})
locationsDF['locationCount'] = pointCount
locationsDF['locationID'] = [x['_id']['locationid'] for x in uniqueLocations]
locationsDF = gpd.GeoDataFrame(locationsDF)

#then add lat lng fields in the df for non Shapely ops
locationsDF['lng'] = locationsDF['geometry'].map(lambda x: x.xy[0][0])
locationsDF['lat'] = locationsDF['geometry'].map(lambda x: x.xy[1][0])

NameError: name 'pointCount' is not defined